In [1]:
import arcpy
from datetime import datetime as d
import os

In [3]:
print(r"start")
import arcpy

#setting worksplace for files
arcpy.env.workspace = r"C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\florida_economic_analysis_2.gdb"

#MakeXYEventLayer - converts csv with lat and lon into a point feature class in ArcGIS which I call the 'florida_houses' feature
#table_name = r"C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\data\FL_coastal_mangrove_2ndbatch.csv"
table_name = r"C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\data\trimmed_second_coastal_batch.csv"
arcpy.conversion.ExportTable(
    in_table=table_name,
    out_table=r"C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\florida_economic_analysis_2.gdb\florida_table_temp",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'transid "transid" true true false 4 Long 0 0,First,#,C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\data\FL_coastal_mangrove_propertyfe.csv,transid,-1,-1;propertyaddresslatitude "propertyaddresslatitude" true true false 8 Double 0 0,First,#,C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\data\FL_coastal_mangrove_propertyfe.csv,propertyaddresslatitude,-1,-1;propertyaddresslongitude "propertyaddresslongitude" true true false 8 Double 0 0,First,#,C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\data\FL_coastal_mangrove_propertyfe.csv,propertyaddresslongitude,-1,-1',
    sort_field=None
)

#arcpy.conversion.ExportTable(table_name, r"C:\Users\brook\Documents\ArcGIS\Projects\florida_economic_analysis\florida_economic_analysis.gdb\florida_table_temp", '', "NOT_USE_ALIAS", 'TransId "TransId" true true false 4 Long 0 0,First,#,FL_coastal_mangrove.csv,TransId,-1,-1;PropertyAddressLatitude "PropertyAddressLatitude" true true false 8000 Float 0 0,First,#,ZTrans_PropertyInfo_12011_20220429.csv,PropertyAddressLatitude,0,8000;PropertyAddressLongitude "PropertyAddressLongitude" true true false 8000 Float 0 0,First,#,ZTrans_PropertyInfo_12011_20220429.csv,PropertyAddressLongitude,0,8000', None)
print("arcpy.conversion.ExportTable = complete!")
arcpy.management.MakeXYEventLayer(table = "florida_table_temp", in_y_field = "PropertyAddressLatitude",  in_x_field = "PropertyAddressLongitude", out_layer = "florida_houses")
print("arcpy.management.MakeXYEventLayer = complete!")

print("***block done***")

start
arcpy.conversion.ExportTable = complete!
arcpy.management.MakeXYEventLayer = complete!
***block done***


In [4]:
from datetime import datetime as d
#joining the housing data with the county data and preparing to join it with elevation data
arcpy.analysis.SpatialJoin("florida_houses", "tl_2016_12_cousub", "florida_houses_county_elevation", "JOIN_ONE_TO_ONE", "KEEP_COMMON", 'TransId "TransId" true true false 4 Long 0 0,First,#,florida_houses,TransId,-1,-1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,tl_2016_12_cousub,COUNTYFP,0,3;COUSUBFP "COUSUBFP" true true false 5 Text 0 0,First,#,tl_2016_12_cousub,COUSUBFP,0,5', "WITHIN", None, '')
print("arcpy.analysis.SpatialJoin - country = complete!", print(d.now()))

#joining the housing w/ county data with the elevation data
arcpy.sa.ExtractMultiValuesToPoints("florida_houses_county_elevation", "USGS_13_2020_2022_Florida_10m_DEM elevation_meters", "NONE")
#arcpy.management.AlterField("florida_houses_county_elevation", "grid_code", "elevation_m") #renaming fields for clarity
print("arcpy.sa.ExtractMultiValuesToPoints - elevation = complete!", print(d.now()))

#extracting mangrove quality data from the raster into point data for a spatial join later, also renaming field names for clarity

#joining the housing data with shoreline distance
fc_target = "florida_houses_county_elevation"
fc_join = "NHDFlowline_Florida"
out_fc = "florida_houses_county_elevation_coast"
arcpy.SpatialJoin_analysis(fc_target,fc_join, out_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Closest geodesic", "", "coastline_dist")
print("arcpy.analysis.SpatialJoin - florida_houses_county_elevation_coast = complete!", print(d.now()))

print("***block done***")


2023-03-09 15:00:51.874512
arcpy.analysis.SpatialJoin - country = complete! None
2023-03-09 15:01:39.980060
arcpy.sa.ExtractMultiValuesToPoints - elevation = complete! None
2023-03-09 16:52:13.548395
arcpy.analysis.SpatialJoin - florida_houses_county_elevation_coast = complete! None
***block done***


In [ ]:
#skip this please for this next run
arcpy.conversion.RasterToPoint("Mangrove_agb_UnitedStates.tif", "Mangrove_agb_UnitedStates", "Value")
print("arcpy.conversion.RasterToPoint - agb = complete!", print(d.now()))
arcpy.management.AlterField("Mangrove_agb_UnitedStates", "grid_code", "agb_value", "agb_value")
arcpy.conversion.RasterToPoint("Mangrove_hmax95_UnitedStates.tif", "Mangrove_hmax95_UnitedStates", "Value")
arcpy.management.AlterField("Mangrove_hmax95_UnitedStates", "grid_code", "hmax95_value", "hmax95_value")
print("arcpy.conversion.RasterToPoint - hmax 95 = complete!", print(d.now()))
arcpy.conversion.RasterToPoint("Mangrove_hba95_UnitedStates.tif", "Mangrove_hba95_UnitedStates", "Value")
arcpy.management.AlterField("Mangrove_hba95_UnitedStates", "grid_code", "hba95_value", "hba95_value")
print("arcpy.conversion.RasterToPoint - hba 95 = complete!", print(d.now()))

In [11]:
#skip for now...
#joining the housing data with mangrove above ground biomass (agb)
fc_target = "florida_houses_county_elevation"
fc_join = "Mangrove_agb_UnitedStates"
out_fc = "florida_houses_county_elevation_agb"
arcpy.SpatialJoin_analysis(fc_target,fc_join, out_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Closest geodesic", "", fc_join+"_dist")
#arcpy.analysis.SpatialJoin("florida_houses_county_elevation", "Mangrove_agb_UnitedStates", r"C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\florida_economic_analysis_2.gdb\florida_houses_c_SpatialJoin", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,florida_houses_county_elevation,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,florida_houses_county_elevation,TARGET_FID,-1,-1;TransId "TransId" true true false 4 Long 0 0,First,#,florida_houses_county_elevation,TransId,-1,-1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,florida_houses_county_elevation,COUNTYFP,0,3;COUSUBFP "COUSUBFP" true true false 5 Text 0 0,First,#,florida_houses_county_elevation,COUSUBFP,0,5;elevation_meters "elevation_meters" true true false 4 Float 0 0,First,#,florida_houses_county_elevation,elevation_meters,-1,-1;pointid "pointid" true true false 4 Long 0 0,First,#,Mangrove_agb_UnitedStates,pointid,-1,-1;agb_value "agb_value" true true false 4 Float 0 0,First,#,Mangrove_agb_UnitedStates,agb_value,-1,-1', "CLOSEST_GEODESIC", None, "agb_dist")
print("arcpy.analysis.SpatialJoin - florida_houses_elevation_agb = complete!", print(d.now()))

#joining the housing data with basal area weighted height (Hba95)
fc_target = "florida_houses_county_elevation_agb"
fc_join = "Mangrove_hba95_UnitedStates"
out_fc = "florida_houses_county_elevation_agb_hba95"
arcpy.SpatialJoin_analysis(fc_target,fc_join, out_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Closest geodesic", "", fc_join+"_dist")
print("arcpy.analysis.SpatialJoin - florida_houses_county_elevation_agb_hba95 = complete!", print(d.now()))

#joining the housing data with mangrove height (Hmax95)
fc_target = "florida_houses_county_elevation_agb_hba95"
fc_join = "Mangrove_hmax95_UnitedStates"
out_fc = "florida_houses_county_elevation_agb_hba95_hmax95"
arcpy.SpatialJoin_analysis(fc_target,fc_join, out_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Closest geodesic", "", fc_join+"_dist")
print("arcpy.analysis.SpatialJoin - florida_houses_county_elevation_agb_hba95_hmax95 = complete!", print(d.now()))

#joining the housing data with shoreline distance
fc_target = "florida_houses_county_elevation_agb_hba95_hmax95"
fc_join = "NHDFlowline_Florida"
out_fc = "florida_houses_county_elevation_agb_hba95_hmax95_coast"
arcpy.SpatialJoin_analysis(fc_target,fc_join, out_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Closest geodesic", "", "coastline_dist")
print("arcpy.analysis.SpatialJoin - florida_houses_county_elevation_agb_hba95_hmax95_coast = complete!", print(d.now()))

print("***block done***")

2023-01-30 02:40:20.142989
arcpy.analysis.SpatialJoin - florida_houses_elevation_agb = complete! None
2023-01-30 02:47:07.929812
arcpy.analysis.SpatialJoin - florida_houses_county_elevation_agb_hba95 = complete! None
2023-01-30 02:53:58.652418
arcpy.analysis.SpatialJoin - florida_houses_county_elevation_agb_hba95_hmax95 = complete! None
2023-01-30 05:03:38.852564
arcpy.analysis.SpatialJoin - florida_houses_county_elevation_agb_hba95_hmax95_coast = complete! None
***block done***


In [5]:
#from arcpy import env
#from datetime import datetime as d
startTime = d.now()
print("starting code...")
arcpy.env.workspace = r"C:\Users\Administrator\Documents\ArcGIS\Packages\florida_economic_analysis_69b365\p30\florida_economic_analysis.gdb"
#target_feature = r"C:\Users\brook\Documents\ArcGIS\Projects\florida_economic_analysis\florida_economic_analysis.gdb\florida_houses_county_elevation_agb_hba95_hmax95_coast"
target_feature = "florida_houses_county_elevation_coast"
#target_feature = "florida_houses_county_elevation_agb_hba95_hmax95_coast"
#print(target_feature)

arcpy.env.overwriteOutput = True
fcs = arcpy.ListFeatureClasses("gmw_v3*") #gets the feature classes 
print("fcs = ")
print(fcs)
print("target_feature = ")
print(target_feature)

counter = 1
fc1 = target_feature
for fc in fcs:
    print(fc, print(d.now()))
    #out_fc = "SP_Test" + str(counter)
    out_fc = "SP_Join_" + str(fc)
    arcpy.SpatialJoin_analysis(fc1,fc, out_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Closest geodesic", "", str(out_fc))
    counter += 1
    fc1 = out_fc
print ('(Elapsed time: ' + str(d.now() - startTime)[:-3] + ')')

starting code...
fcs = 
['gmw_v3_2008_vec', 'gmw_v3_2009_vec', 'gmw_v3_1996_vec', 'gmw_v3_2007_vec', 'gmw_v3_2010_vec', 'gmw_v3_2015_vec', 'gmw_v3_2016_vec', 'gmw_v3_2017_vec', 'gmw_v3_2018_vec', 'gmw_v3_2019_vec', 'gmw_v3_2020_vec']
target_feature = 
florida_houses_county_elevation_coast
2023-03-09 16:52:13.976392
gmw_v3_2008_vec None
2023-03-09 18:47:39.574306
gmw_v3_2009_vec None
2023-03-09 20:40:46.261228
gmw_v3_1996_vec None
2023-03-09 22:36:03.983547
gmw_v3_2007_vec None
2023-03-10 00:26:54.267886
gmw_v3_2010_vec None
2023-03-10 02:21:15.448153
gmw_v3_2015_vec None
2023-03-10 04:12:41.654795
gmw_v3_2016_vec None
2023-03-10 06:03:04.441076
gmw_v3_2017_vec None
2023-03-10 07:52:44.035361
gmw_v3_2018_vec None
2023-03-10 09:41:40.117508
gmw_v3_2019_vec None
2023-03-10 11:30:02.134918
gmw_v3_2020_vec None
(Elapsed time: 20:26:19.202)


In [5]:
#this returns the distance between each house to the GMW mangrove quality data which is different than the mangrove years distance

#arcpy.analysis.SpatialJoin("SP_Join_gmw_v3_2020_vec", "Mangrove_agb_UnitedStates", r"C:\Users\Administrator\Documents\ArcGIS\Packages\florida_economic_analysis_69b365\p30\florida_economic_analysis_2.gdb\mangrove_all_join_20230121", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'SP_Join_gmw_v3_2020_vec "SP_Join_gmw_v3_2020_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2020_vec,-1,-1;SP_Join_gmw_v3_2019_vec "SP_Join_gmw_v3_2019_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2019_vec,-1,-1;SP_Join_gmw_v3_2018_vec "SP_Join_gmw_v3_2018_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2018_vec,-1,-1;SP_Join_gmw_v3_2017_vec "SP_Join_gmw_v3_2017_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2017_vec,-1,-1;SP_Join_gmw_v3_2016_vec "SP_Join_gmw_v3_2016_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2016_vec,-1,-1;SP_Join_gmw_v3_2015_vec "SP_Join_gmw_v3_2015_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2015_vec,-1,-1;SP_Join_gmw_v3_2010_vec "SP_Join_gmw_v3_2010_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2010_vec,-1,-1;SP_Join_gmw_v3_2009_vec "SP_Join_gmw_v3_2009_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2009_vec,-1,-1;SP_Join_gmw_v3_2008_vec "SP_Join_gmw_v3_2008_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2008_vec,-1,-1;SP_Join_gmw_v3_2007_vec "SP_Join_gmw_v3_2007_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2007_vec,-1,-1;SP_Join_gmw_v3_1996_vec "SP_Join_gmw_v3_1996_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_1996_vec,-1,-1;coastline_dist "coastline_dist" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,coastline_dist,-1,-1;TransId "TransId" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TransId,-1,-1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,COUNTYFP,0,3;COUSUBFP "COUSUBFP" true true false 5 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,COUSUBFP,0,5;elevation_meters "elevation_meters" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,elevation_meters,-1,-1;agb_value "agb_value" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,agb_value,-1,-1;hba95_value "hba95_value" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,hba95_value,-1,-1;hmax95_value "hmax95_value" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,hmax95_value,-1,-1;pointid "pointid" true true false 4 Long 0 0,First,#,Mangrove_agb_UnitedStates,pointid,-1,-1', "CLOSEST_GEODESIC", None, "mangrove_quality_distance_meters")
arcpy.analysis.SpatialJoin("SP_Join_gmw_v3_2020_vec", "Mangrove_agb_UnitedStates", r"C:\Users\Administrator\Documents\ArcGIS\Projects\florida_economic_analysis\florida_economic_analysis_2.gdb\mangrove_all_join_20230217", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count,-1,-1;SP_Join_gmw_v3_2020_vec "SP_Join_gmw_v3_2020_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2020_vec,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_1,-1,-1;SP_Join_gmw_v3_2019_vec "SP_Join_gmw_v3_2019_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2019_vec,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12,-1,-1;SP_Join_gmw_v3_2018_vec "SP_Join_gmw_v3_2018_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2018_vec,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13,-1,-1;SP_Join_gmw_v3_2017_vec "SP_Join_gmw_v3_2017_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2017_vec,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13,-1,-1;Join_Count_12_13_14 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14,-1,-1;SP_Join_gmw_v3_2016_vec "SP_Join_gmw_v3_2016_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2016_vec,-1,-1;TARGET_FID_12_13_14 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14,-1,-1;Join_Count_12_13_14_15 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15,-1,-1;SP_Join_gmw_v3_2015_vec "SP_Join_gmw_v3_2015_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2015_vec,-1,-1;TARGET_FID_12_13_14_15 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15,-1,-1;Join_Count_12_13_14_15_16 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16,-1,-1;SP_Join_gmw_v3_2010_vec "SP_Join_gmw_v3_2010_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2010_vec,-1,-1;TARGET_FID_12_13_14_15_16 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16,-1,-1;Join_Count_12_13_14_15_16_17 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17,-1,-1;SP_Join_gmw_v3_2007_vec "SP_Join_gmw_v3_2007_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2007_vec,-1,-1;TARGET_FID_12_13_14_15_16_17 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17,-1,-1;Join_Count_12_13_14_15_16_17_18 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18,-1,-1;SP_Join_gmw_v3_1996_vec "SP_Join_gmw_v3_1996_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_1996_vec,-1,-1;TARGET_FID_12_13_14_15_16_17_18 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18,-1,-1;Join_Count_12_13_14_15_16_17_18_19 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18_19,-1,-1;SP_Join_gmw_v3_2009_vec "SP_Join_gmw_v3_2009_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2009_vec,-1,-1;TARGET_FID_12_13_14_15_16_17_18_19 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18_19,-1,-1;Join_Count_12_13_14_15_16_17_18_19_20 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18_19_20,-1,-1;SP_Join_gmw_v3_2008_vec "SP_Join_gmw_v3_2008_vec" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,SP_Join_gmw_v3_2008_vec,-1,-1;TARGET_FID_12_13_14_15_16_17_18_19_20 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18_19_20,-1,-1;Join_Count_12_13_14_15_16_17_18_19_20_21 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18_19_20_21,-1,-1;coastline_dist "coastline_dist" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,coastline_dist,-1,-1;TARGET_FID_12_13_14_15_16_17_18_19_20_21 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18_19_20_21,-1,-1;Join_Count_12_13_14_15_16_17_18_19_20_21_22 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18_19_20_21_22,-1,-1;Mangrove_hmax95_UnitedStates_dist "Mangrove_hmax95_UnitedStates_dist" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,Mangrove_hmax95_UnitedStates_dist,-1,-1;TARGET_FID_12_13_14_15_16_17_18_19_20_21_22 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18_19_20_21_22,-1,-1;Join_Count_12_13_14_15_16_17_18_19_20_21_22_23 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18_19_20_21_22_23,-1,-1;Mangrove_hba95_UnitedStates_dist "Mangrove_hba95_UnitedStates_dist" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,Mangrove_hba95_UnitedStates_dist,-1,-1;TARGET_FID_12_13_14_15_16_17_18_19_20_21_22_23 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18_19_20_21_22_23,-1,-1;Join_Count_12_13_14_15_16_17_18_19_20_21_22_23_24 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18_19_20_21_22_23_24,-1,-1;Mangrove_agb_UnitedStates_dist "Mangrove_agb_UnitedStates_dist" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,Mangrove_agb_UnitedStates_dist,-1,-1;TARGET_FID_12_13_14_15_16_17_18_19_20_21_22_23_24 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18_19_20_21_22_23_24,-1,-1;Join_Count_12_13_14_15_16_17_18_19_20_21_22_23_24_25 "Join_Count" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Join_Count_12_13_14_15_16_17_18_19_20_21_22_23_24_25,-1,-1;TARGET_FID_12_13_14_15_16_17_18_19_20_21_22_23_24_25 "TARGET_FID" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TARGET_FID_12_13_14_15_16_17_18_19_20_21_22_23_24_25,-1,-1;TransId "TransId" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,TransId,-1,-1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,COUNTYFP,0,3;COUSUBFP "COUSUBFP" true true false 5 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,COUSUBFP,0,5;elevation_meters "elevation_meters" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,elevation_meters,-1,-1;pointid "pointid" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,pointid,-1,-1;agb_value "agb_value" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,agb_value,-1,-1;pointid_1 "pointid" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,pointid_1,-1,-1;hba95_value "hba95_value" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,hba95_value,-1,-1;pointid_12 "pointid" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,pointid_12,-1,-1;hmax95_value "hmax95_value" true true false 4 Float 0 0,First,#,SP_Join_gmw_v3_2020_vec,hmax95_value,-1,-1;Permanent_Identifier "Permanent_Identifier" true true false 40 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,Permanent_Identifier,0,40;FDate "FDate" true true false 8 Date 0 0,First,#,SP_Join_gmw_v3_2020_vec,FDate,-1,-1;Resolution "Resolution" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,Resolution,-1,-1;GNIS_ID "GNIS_ID" true true false 10 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,GNIS_ID,0,10;GNIS_Name "GNIS_Name" true true false 65 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,GNIS_Name,0,65;LengthKM "LengthKM" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,LengthKM,-1,-1;ReachCode "ReachCode" true true false 14 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,ReachCode,0,14;FlowDir "FlowDir" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,FlowDir,-1,-1;WBArea_Permanent_Identifier "WBArea_Permanent_Identifier" true true false 40 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,WBArea_Permanent_Identifier,0,40;FType "FType" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,FType,-1,-1;FCode "FCode" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,FCode,-1,-1;MainPath "MainPath" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,MainPath,-1,-1;InNetwork "InNetwork" true true false 2 Short 0 0,First,#,SP_Join_gmw_v3_2020_vec,InNetwork,-1,-1;VisibilityFilter "VisibilityFilter" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,VisibilityFilter,-1,-1;NHDPlusID "NHDPlusID" true true false 8 Double 0 0,First,#,SP_Join_gmw_v3_2020_vec,NHDPlusID,-1,-1;VPUID "VPUID" true true false 8 Text 0 0,First,#,SP_Join_gmw_v3_2020_vec,VPUID,0,8;Enabled "Enabled" true true false 2 Short 0 0,First,#,SP_Join_gmw_v3_2020_vec,Enabled,-1,-1;PXLVAL "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL,-1,-1;PXLVAL_1 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_1,-1,-1;PXLVAL_12 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12,-1,-1;PXLVAL_12_13 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13,-1,-1;PXLVAL_12_13_14 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13_14,-1,-1;PXLVAL_12_13_14_15 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13_14_15,-1,-1;PXLVAL_12_13_14_15_16 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13_14_15_16,-1,-1;PXLVAL_12_13_14_15_16_17 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13_14_15_16_17,-1,-1;PXLVAL_12_13_14_15_16_17_18 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13_14_15_16_17_18,-1,-1;PXLVAL_12_13_14_15_16_17_18_19 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13_14_15_16_17_18_19,-1,-1;PXLVAL_12_13_14_15_16_17_18_19_20 "PXLVAL" true true false 4 Long 0 0,First,#,SP_Join_gmw_v3_2020_vec,PXLVAL_12_13_14_15_16_17_18_19_20,-1,-1;pointid_12_13 "pointid" true true false 4 Long 0 0,First,#,Mangrove_agb_UnitedStates,pointid,-1,-1;agb_value_1 "agb_value" true true false 4 Float 0 0,First,#,Mangrove_agb_UnitedStates,agb_value,-1,-1', "CLOSEST_GEODESIC", None, "mangrove_quality_distance")


<Result 'C:\\Users\\Administrator\\Documents\\ArcGIS\\Projects\\florida_economic_analysis\\florida_economic_analysis_2.gdb\\mangrove_all_join_20230203'>

In [ ]:
#joining the housing data with shoreline distance
#you'll need to run this once rather quickly...
#please delete after done.
#fc_target = "florida_houses_county_elevation"#fix this letter here
fc_target = "SP_Join_gmw_v3_2020_vec"
fc_join = "NHDFlowline_Florida"
out_fc = "florida_houses_county_elevation_coast"
arcpy.SpatialJoin_analysis(fc_target,fc_join, out_fc, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Closest geodesic", "", "coastline_dist")
print("arcpy.analysis.SpatialJoin - florida_houses_county_elevation_coast = complete!", print(d.now()))

print("***block done***")